In [1]:
# Cell 1
#ae_rf_fusion.ipynb

%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd

# Setup path
current_dir = Path.cwd()
root_dir = current_dir.parent
if str(root_dir) not in sys.path:
    sys.path.append(str(root_dir))

print(f"Project Root: {root_dir}")

# Import modules
from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils
from sklearn.model_selection import train_test_split

# Kiểm tra đường dẫn Data Folder thay vì Data Path
print(f"Checking Data Directories:")
print(f"  - 2017: {config.DIR_2017} -> {'✅ Có' if config.DIR_2017.exists() else '❌ Không thấy'}")
print(f"  - 2018: {config.DIR_2018} -> {'✅ Có' if config.DIR_2018.exists() else '❌ Không thấy'}")

if not config.DIR_2017.exists() or not config.DIR_2018.exists():
    print("⚠️ CẢNH BÁO: Hãy kiểm tra lại đường dẫn folder trong src/config.py")

# --- KHỞI TẠO EXPERIMENT MỚI ---
# Bước này sẽ tạo folder dạng: results/experiments/exp_20251212_064500/
exp_paths = utils.setup_experiment_folder()
print(f"🚀 Experiment ID initialized.")

Project Root: c:\Users\Admin\Documents\ids_ae_rf_hybrid
Checking Data Directories:
  - 2017: C:\Users\Admin\Documents\ids_ae_rf_hybrid\datasets\CIC-IDS2017 -> ✅ Có
  - 2018: C:\Users\Admin\Documents\ids_ae_rf_hybrid\datasets\CSE-CIC-IDS2018 -> ✅ Có
📂 New Experiment Created: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251217_233222
🚀 Experiment ID initialized.


In [2]:
# Cell 2: Load Data (Mixed & Chunked)
try:
    print("🚀 Bắt đầu load dữ liệu hỗn hợp (Chunking Mode)...")
    
    # Hàm này sẽ tự động đọc chunk, lọc cột, gộp 2017+2018
    # Đảm bảo bạn đã cập nhật src/preprocessing.py theo code "Chunking"
    
    X, y = preprocessing.load_mixed_datasets_chunked(binary_mode=True)



    
    
    print("✅ Load xong. Sẵn sàng train!")
except AttributeError:
    print("❌ Lỗi: Không tìm thấy hàm 'load_mixed_datasets_chunked'.")
    print("👉 Hãy chắc chắn bạn đã cập nhật file src/preprocessing.py mới nhất!")
except Exception as e:
    print(f"❌ Lỗi: {e}")

🚀 Bắt đầu load dữ liệu hỗn hợp (Chunking Mode)...
🔄 Loading mixed datasets (Chunking Mode, Binary=True)...
✅ Loaded Mixed Total: (12455891, 67)
   Labels distribution:
Label
0    9150010
1    3305881
Name: count, dtype: int64
✅ Load xong. Sẵn sàng train!


In [3]:
# Cell: Sanity Check - Kiểm tra Chuẩn hóa và Thứ tự
import pandas as pd
import numpy as np
from src import config, preprocessing

def verify_feature_order_and_normalization():
    print("🕵️♂️ BẮT ĐẦU KIỂM TRA TÍNH TOÀN VẸN DỮ LIỆU...\n")
    
    # 1. Kiểm tra Thứ tự (Order Guarantee)
    # ---------------------------------------------------------
    print("1️⃣  Kiểm tra Config (Khuôn mẫu):")
    expected_cols = config.SELECTED_FEATURES
    print(f"   - Số lượng features trong Config: {len(expected_cols)}")
    print(f"   - 5 features đầu: {expected_cols[:5]}")
    print(f"   - 5 features cuối: {expected_cols[-5:]}")
    print("   ✅ Kết luận: Biến X (numpy) của bạn CHẮC CHẮN tuân theo thứ tự này.")
    print("      (Vì code load dùng lệnh: chunk[config.SELECTED_FEATURES])\n")

    # 2. Kiểm tra Chuẩn hóa trên File Thực (Real Data Test)
    # ---------------------------------------------------------
    print("2️⃣  Kiểm tra thực tế trên file CSV (Mỗi năm 1 file):")
    
    # Lấy 1 file mẫu từ mỗi năm
    files_to_check = {
        "2017": next(config.DIR_2017.glob("*.csv"), None),
        "2018": next(config.DIR_2018.glob("*.csv"), None)
    }

    for year, file_path in files_to_check.items():
        if not file_path:
            print(f"   ⚠️ Không tìm thấy file {year}")
            continue
            
        print(f"\n   👉 Kiểm tra dataset {year}: {file_path.name}")
        
        # Đọc thô 5 dòng
        df_raw = pd.read_csv(file_path, nrows=5)
        print(f"      - Cột gốc (Raw): {df_raw.columns.tolist()[:3]} ...")
        
        # Áp dụng hàm chuẩn hóa
        df_norm = preprocessing.normalize_column_names(df_raw)
        
        # Kiểm tra xem các cột quan trọng có xuất hiện không
        # Ví dụ: Kiểm tra xem cột 'Destination Port' (đã chuẩn hóa) có mặt không
        target_col = "Destination Port"
        if target_col in df_norm.columns:
            print(f"      - ✅ Đã tìm thấy '{target_col}' (Chuẩn hóa OK)")
        else:
            print(f"      - ❌ CẢNH BÁO: Không thấy '{target_col}'. Có thể mapping tên bị sai!")
            
        # Kiểm tra sự khớp lệnh với Config
        # Tạo df giả lập bước lọc
        for col in config.SELECTED_FEATURES:
            if col not in df_norm.columns:
                df_norm[col] = 0.0 # Giả lập fill missing
        
        df_final = df_norm[config.SELECTED_FEATURES]
        
        # So sánh thứ tự
        if df_final.columns.tolist() == config.SELECTED_FEATURES:
            print(f"      - ✅ Thứ tự cột sau khi lọc khớp 100% với Config.")
        else:
            print(f"      - ❌ SAI THỨ TỰ! Cần kiểm tra lại code.")

verify_feature_order_and_normalization()

🕵️♂️ BẮT ĐẦU KIỂM TRA TÍNH TOÀN VẸN DỮ LIỆU...

1️⃣  Kiểm tra Config (Khuôn mẫu):
   - Số lượng features trong Config: 67
   - 5 features đầu: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets']
   - 5 features cuối: ['Active Min', 'Idle Mean', 'Idle Std', 'Idle Max', 'Idle Min']
   ✅ Kết luận: Biến X (numpy) của bạn CHẮC CHẮN tuân theo thứ tự này.
      (Vì code load dùng lệnh: chunk[config.SELECTED_FEATURES])

2️⃣  Kiểm tra thực tế trên file CSV (Mỗi năm 1 file):

   👉 Kiểm tra dataset 2017: Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv
      - Cột gốc (Raw): [' Destination Port', ' Flow Duration', ' Total Fwd Packets'] ...
      - ✅ Đã tìm thấy 'Destination Port' (Chuẩn hóa OK)
      - ✅ Thứ tự cột sau khi lọc khớp 100% với Config.

   👉 Kiểm tra dataset 2018: Bot.csv
      - Cột gốc (Raw): ['Dst Port', 'Protocol', 'Flow Duration'] ...
      - ✅ Đã tìm thấy 'Destination Port' (Chuẩn hóa OK)
      - ✅ Thứ tự cột sau k

In [4]:
# Cell 3: Chia tập Train/Test & Scale
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=config.SEED, stratify=y
)

In [5]:
# Cell 4: Scaling & Save Scaler
print("Scaling data...")

scaler = preprocessing.get_scaler()

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test) # Quan trọng: Test phải scale theo Train

print("Done scaling.")


# --- LƯU SCALER VÀO FOLDER EXPERIMENT ---
import joblib

print("💾 Saving scaler...")
scaler_path = exp_paths["models"] / "scaler.joblib"  # 
joblib.dump(scaler, scaler_path)
print(f"✅ Saved scaler: {scaler_path}")

Scaling data...
Done scaling.
💾 Saving scaler...
✅ Saved scaler: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251217_233222\models\scaler.joblib


In [6]:
# Cell 5: 
print(f"Training Autoencoder on: {config.DEVICE}")

# 1. Initialize model
ae_model = autoencoder.DeepAutoencoder(         # hyperparameters from config.py
    input_dim=config.AE_INPUT_DIM,              # 67
    latent_dim=config.AE_LATENT_DIM             # 20
    ,hidden_layers=config.AE_HIDDEN_LAYERS      # [48,32]
)

ae_save_path = exp_paths["models"] / "ae_model.pth"

# 2. Train model
ae_model = autoencoder.train_ae(ae_model, X_train_scaled, save_path=ae_save_path)

print("AE training done!")

Training Autoencoder on: cuda
[Autoencoder] Training on cuda...
  Epoch 5/20 - Loss: 0.170112
  Epoch 10/20 - Loss: 0.120580
  Epoch 15/20 - Loss: 0.098475
  Epoch 20/20 - Loss: 0.087598
✅ AE Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251217_233222\models\ae_model.pth
AE training done!


In [7]:
# Cell 6
# Extract Latent Features Z
print("Extracting Latent Features (Z) ...")
X_train_latent = autoencoder.extract_features(ae_model, X_train_scaled)
X_test_latent = autoencoder.extract_features(ae_model, X_test_scaled)

print(f"Original shape: {X_train_scaled.shape}") # Original shape
print(f"Latent shape:   {X_train_latent.shape}") # Latent shape


Extracting Latent Features (Z) ...
Original shape: (9964712, 67)
Latent shape:   (9964712, 20)


In [8]:
import gc

# 1. Xóa các biến dữ liệu cũ để giải phóng RAM
print("🗑️ Cleaning up RAM...")
try:
    del X
    del X_train
    del X_test
    # Nếu có các biến phụ khác thì xóa nốt
except:
    pass

# 2. Ép Python thu hồi bộ nhớ ngay lập tức
gc.collect()

print("✅ RAM Cleaned.")

🗑️ Cleaning up RAM...
✅ RAM Cleaned.


In [10]:
# Cell 7: Chuẩn bị dữ liệu cho Fusion

# 1. Lấy danh sách tên cột đầy đủ
all_feature_names = config.SELECTED_FEATURES # List 67 cột

# 2. Tìm vị trí index của 20 cột mRMR trong list 67 cột
try:
    mrmr_indices = [all_feature_names.index(feat) for feat in config.mRMR_FEATURES]
    print(f"✅ Found indices for {len(mrmr_indices)} mRMR features.")
except ValueError as e:
    print(f"❌ Error: Feature name not found in SELECTED_FEATURES. Check config.py!\nDetails: {e}")
    raise e

# 3. Cắt dữ liệu (Slicing) từ tập đã Scale
# Chỉ lấy các cột nằm trong mrmr_indices
print("✂️ Slicing mRMR features from Scaled Data...")
X_train_mrmr = X_train_scaled[:, mrmr_indices]
X_test_mrmr  = X_test_scaled[:, mrmr_indices]

print(f"X_train_mrmr shape: {X_train_mrmr.shape}") # MONG ĐỢI: (N, 20)

# (Lưu ý: Bạn đã extract latent ở Cell 6 rồi, không cần chạy lại dòng extract ở đây nữa để tiết kiệm thời gian,
# nhưng nếu bạn muốn chắc chắn thì cứ để nguyên dòng extract ở dưới cũng không sao)
# print("🧠 Extracting Latent Features...")
# X_train_latent = autoencoder.extract_features(ae_model, X_train_scaled)
# X_test_latent  = autoencoder.extract_features(ae_model, X_test_scaled)

✅ Found indices for 20 mRMR features.
✂️ Slicing mRMR features from Scaled Data...
X_train_mrmr shape: (9964712, 20)


In [11]:
# Cell 8: Fusion 
import numpy as np

print("🔗 Fusing: [mRMR 20] + [Latent 20] ...")

# --- SỬA LỖI Ở ĐÂY ---
# Cũ (Sai): np.hstack([X_train_scaled, X_train_latent]) -> Ra 87 cột
# Mới (Đúng): np.hstack([X_train_mrmr, X_train_latent]) -> Ra 40 cột

X_train_fusion = np.hstack([X_train_mrmr, X_train_latent])
X_test_fusion  = np.hstack([X_test_mrmr, X_test_latent])

print(f"🔥 Final Fusion Input Shape: {X_train_fusion.shape}")
# Kiểm tra nhanh: Nếu shape[1] != 40 thì báo động
if X_train_fusion.shape[1] != 40:
    print(f"⚠️ WARNING: Shape không phải 40! Đang là {X_train_fusion.shape[1]}")
else:
    print("✅ Shape chuẩn 40 features (20 mRMR + 20 Latent).")

# Dọn dẹp RAM
try:
    del X_train_scaled
    del X_test_scaled
    # Xóa luôn biến mrmr trung gian nếu muốn
    del X_train_mrmr
    del X_test_mrmr
    gc.collect()
    print("✅ Scaled Data removed after fusion.")
except:
    pass

🔗 Fusing: [mRMR 20] + [Latent 20] ...
🔥 Final Fusion Input Shape: (9964712, 40)
✅ Shape chuẩn 40 features (20 mRMR + 20 Latent).
✅ Scaled Data removed after fusion.


In [12]:
# Cell 9: Train RF on Fusion Data
print("🚀 Starting RF Training on Fusion Data...")

rf_save_path = exp_paths["models"] / "rf_model.joblib"

rf_model = rf_classifier.train_rf(X_train_fusion, y_train, save_path=rf_save_path)

print("RF training done!")

🚀 Starting RF Training on Fusion Data...
[RandomForest] Training classifier...


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100


[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:  8.5min


building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79

[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed: 22.8min finished


✅ RF Model saved to: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251217_233222\models\rf_model.joblib
RF training done!


In [13]:
# Cell 10: Evaluate RF on Fusion Data

label_names = ["Normal", "Attack"] if config.BINARY_MODE else None

print("\n--- STARTING EVALUATION ---")

# Đánh giá và vẽ Confusion Matrix
metrics_result = evaluation.evaluate_model(
    model=rf_model, 
    X_test=X_test_fusion, 
    y_test=y_test, 
    save_dir=exp_paths["figures"], 
    dataset_name="Mixed Test Set"
)

# In tóm tắt kết quả
print("\n--- SUMMARY ---")
print(f"Accuracy: {metrics_result['accuracy']:.4f}")
print(f"MCC:      {metrics_result['mcc']:.4f}")
if 'recall' in metrics_result: # Kiểm tra key để tránh lỗi
    print(f"Recall:   {metrics_result['recall']:.4f}")
print(f"📂 Image saved at: {exp_paths['figures']}")


--- STARTING EVALUATION ---

📊 Evaluating on Mixed Test Set...


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    1.9s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    4.8s finished


   ✅ Accuracy: 0.9802
   ⭐ MCC:      0.9493
   📝 Report saved to: report_Mixed_Test_Set.txt
   🖼️ Confusion Matrix saved to: cm_Mixed_Test_Set.png

--- SUMMARY ---
Accuracy: 0.9802
MCC:      0.9493
📂 Image saved at: C:\Users\Admin\Documents\ids_ae_rf_hybrid\results\experiments\exp_20251217_233222\figures
